<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/main/HPfilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_datareader as pdr
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot

# set the start and end dates for the data
start_date = '1994-01-01'
end_date = '2023-01-01'

# download the data from FRED using pandas_datareader
gdp_us = web.DataReader('GDPC1', 'fred', start_date, end_date)
gdp_jp = web.DataReader('JPNRGDPEXP', 'fred', start_date, end_date)

log_gdp_us = np.log(gdp_us)
log_gdp_jp = np.log(gdp_jp)

# calculate the quarterly percent change in real GDP
gdp_pct_change_us = gdp_us.pct_change(4)
gdp_pct_change_jp = gdp_jp.pct_change(4)

# apply a Hodrick-Prescott filter to the data to extract the cyclical component
cycle_us, trend_us = sm.tsa.filters.hpfilter(log_gdp_us, lamb=1600)
cycle_jp, trend_jp = sm.tsa.filters.hpfilter(log_gdp_jp, lamb=1600)

# Create an interactive plot using Plotly
fig = go.Figure()
fig2 = go.Figure()

# Add traces for the original GDP data
fig.add_trace(go.Scatter(x=log_gdp_us.index, y=log_gdp_us.values.flatten(), mode='lines', name='US GDP (log)'))
fig2.add_trace(go.Scatter(x=log_gdp_jp.index, y=log_gdp_jp.values.flatten(), mode='lines', name='Japan GDP (log)'))

# Add traces for the trend components
fig.add_trace(go.Scatter(x=trend_us.index, y=trend_us.values.flatten(), mode='lines', name='US Trend'))
fig2.add_trace(go.Scatter(x=trend_jp.index, y=trend_jp.values.flatten(), mode='lines', name='Japan Trend'))

# Update the layout
fig.update_layout(title='US GDP and Business Cycle Trend',xaxis_title='Date',yaxis_title='GDP (log)',hovermode='x unified')
fig2.update_layout(title='JP GDP and Business Cycle Trend',xaxis_title='Date',yaxis_title='GDP (log)',hovermode='x unified')

# Save the interactive plots as HTML files
plot(fig, filename='us_gdp_plot.html')
plot(fig2, filename='jp_gdp_plot.html')

# Display the plots
fig.show()
fig2.show()

In [4]:
# Create an interactive plot using Plotly
fig3 = go.Figure()

# Add traces for the cyclical components
fig3.add_trace(go.Scatter(x=cycle_us.index, y=cycle_us.values.flatten(), mode='lines', name='US Cycle'))
fig3.add_trace(go.Scatter(x=cycle_jp.index, y=cycle_jp.values.flatten(), mode='lines', name='Japan Cycle'))

# Update the layout
fig3.update_layout(title='US and Japan GDP Cycle Components',xaxis_title='Date',yaxis_title='Cycle Component',hovermode='x unified')

# Display the plot
fig3.show()

# Analyze the cycle components
cycle_us_mean = cycle_us.mean()
cycle_us_std = cycle_us.std()
print("US Cycle Mean:", cycle_us_mean)
print("US Cycle Standard Deviation:", cycle_us_std)

cycle_jp_mean = cycle_jp.mean()
cycle_jp_std = cycle_jp.std()
print("\nJapan Cycle Mean:", cycle_jp_mean)
print("Japan Cycle Standard Deviation:", cycle_jp_std)

corr_coef = cycle_us.corr(cycle_jp)
print(f"\nCorrelation coefficient between US and Japan cycle components: {corr_coef}")

US Cycle Mean: -3.553928281767612e-13
US Cycle Standard Deviation: 0.013354141433165642

Japan Cycle Mean: -1.2762440676921798e-13
Japan Cycle Standard Deviation: 0.01613207731471153

Correlation coefficient between US and Japan cycle components: 0.6919724227584749
